In [1]:
#API_KEY = "874fc67d-cee6-4294-b799-18e2b4e63fb8"  #ez a mienk
API_KEY = "ad6669df-65b6-45f9-8e02-7ba74e788acd"  #ez nem a mienk

Below you can find Python and R code snippets showing how to programmatically interact with the REST API by means of YOUR_API_KEY.

⇒ Query the REST API with your API key:  the code below shows how to retrieve gene-disease associations related to the gene APP amyloid beta precursor protein with NCBI ID equal to 351 (gene_ncbi_id parameter equal to 351). To this purpose we call the gene-disease associations endpoint (i.e. gda endpoint, see the REST API Interactive console for more details). We retrieve the top-100 gene-disease associations involving such gene, ordered by descending gene-disease score, by asking for the first page of results (page_number parameter equal to 0).

In [7]:
import requests
import json
import time


# Specify query parameters by means of a dictionary 
params = {}
# ...retrieve disease associated to gene with NCBI ID 
#params["gene_ncbi_id"] = "351"
params["disease"] = "DO_987"
# ...retrieve the first page of results (page number 0) 
params["page_number"] = "0"

# Create a dictionary with the HTTP headers of your API call 
HTTPheadersDict = {}
# Set the 'Authorization' HTTP header equal to API_KEY (your API key) 
HTTPheadersDict['Authorization'] = API_KEY
# Set the 'accept' HTTP header to specify the response format: one of 'application/json', 'application/xml', 'application/csv' 
HTTPheadersDict['accept'] = 'application/json'

# Query the gda summary endpoint 
response = requests.get("https://api.disgenet.com/api/v1/gda/summary",\
                        params=params, headers=HTTPheadersDict, verify=False)

# If the status code of response is 429, it means you have reached one of your query limits 
# You can retrieve the time you need to wait until doing a new query in the response headers 
if not response.ok:
    if response.status_code == 429:
        while response.ok is False:
            print("You have reached a query limit for your user. Please wait {} seconds until next query".format(\
            response.headers['x-rate-limit-retry-after-seconds']))
            time.sleep(int(response.headers['x-rate-limit-retry-after-seconds']))
            print("Your rate limit is now restored")

            # Repeat your query
            response = requests.get("https://api.disgenet.com/api/v1/gda/summary",\
                                    params=params, headers=HTTPheadersDict, verify=False)
            if response.ok is True:
                break
            else:
                continue

# Parse response content in JSON format since we set 'accept:application/json' as HTTP header 
response_parsed = json.loads(response.text)
print('STATUS: {}'.format(response_parsed["status"]))
print('TOTAL NUMBER OF RESULTS: {}'.format(response_parsed["paging"]["totalElements"]))
print('NUMBER OF RESULTS RETRIEVED BY CURRENT CALL (PAGE NUMBER {}): {}'.format(\
      response_parsed["paging"]["currentPageNumber"], response_parsed["paging"]["totalElementsInPage"]))

c:\Users\Szabi\melytanulas\DisGeNET-BME2024\.venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.disgenet.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


STATUS: OK
TOTAL NUMBER OF RESULTS: 21
NUMBER OF RESULTS RETRIEVED BY CURRENT CALL (PAGE NUMBER 0): 21


In [9]:
import pandas as pd

payload = response_parsed.get('payload', [])

# Convert the payload to a pandas DataFrame
df = pd.DataFrame(payload)

print(df.columns)
print(df.head())


Index(['assocID', 'symbolOfGene', 'geneNcbiID', 'geneEnsemblIDs',
       'geneNcbiType', 'geneDSI', 'geneDPI', 'genepLI', 'geneProteinStrIDs',
       'geneProteinClassIDs', 'geneProteinClassNames', 'diseaseVocabularies',
       'diseaseName', 'diseaseType', 'diseaseUMLSCUI', 'diseaseClasses_MSH',
       'diseaseClasses_UMLS_ST', 'diseaseClasses_DO', 'diseaseClasses_HPO',
       'numCTsupportingAssociation', 'numPMIDs', 'chemicalsIncludedInEvidence',
       'numberPmidsWithChemsIncludedInEvidenceBySource', 'score',
       'yearInitial', 'yearFinal', 'el', 'ei'],
      dtype='object')
   assocID symbolOfGene  geneNcbiID     geneEnsemblIDs    geneNcbiType  \
0  5072651          VDR        7421  [ENSG00000111424]  protein-coding   
1  5072216           HR       55806  [ENSG00000168453]  protein-coding   
2  5071730           AR         367  [ENSG00000169083]  protein-coding   
3  5072450       SRD5A2        6716  [ENSG00000277893]  protein-coding   
4  5072854        FOXN1        8456  [EN